In [1]:
# import sys
# sys.path.append("../PythonScripts")

In [1]:
import numpy as np
import pandas as pd

In [2]:
# from pyspark import SparkContext ,SparkConf

from pyspark.sql import SparkSession
# sc = SparkContext()

In [3]:
spark = SparkSession.builder.getOrCreate()

22/02/19 12:49:11 WARN Utils: Your hostname, machine resolves to a loopback address: 127.0.1.1; using 192.168.31.230 instead (on interface enp0s3)
22/02/19 12:49:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/19 12:49:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/19 12:49:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
import pyspark.sql.functions as sf
from pyspark.sql.types import FloatType

In [5]:
cols= ['User','Recommended','Test']
data = [[1,['prod_1','prod_2','prod_3','prod_4','prod_5'],['prod_10','prod_4','prod_5']],
        [2,['prod_10','prod_12','prod_3','prod_4','prod_15'],['prod_1','prod_4','prod_15']],
        [3,['prod_11','prod_7','prod_8','prod_9','prod_15'],['prod_1','prod_2','prod_3','prod_7','prod_9','prod_11','prod_10','prod_4','prod_15']],
        [4,['prod_9','prod_12','prod_13','prod_14','prod_15'],['prod_12']]]

pdf_rec = pd.DataFrame(data, columns=cols)
print(pdf_rec.dtypes)
pdf_rec.head()

User            int64
Recommended    object
Test           object
dtype: object


,User,Recommended,Test
0,1,"[prod_1, prod_2, prod_3, prod_4, prod_5]","[prod_10, prod_4, prod_5]"
1,2,"[prod_10, prod_12, prod_3, prod_4, prod_15]","[prod_1, prod_4, prod_15]"
2,3,"[prod_11, prod_7, prod_8, prod_9, prod_15]","[prod_1, prod_2, prod_3, prod_7, prod_9, prod_..."
3,4,"[prod_9, prod_12, prod_13, prod_14, prod_15]",[prod_12]


In [6]:
# row1 = [1, ['prod_1','prod_2','prod_3','prod_4','prod_5'], ['prod_1','prod_4','prod_5']]
# row2 = [1, ['prod_1','prod_12','prod_13','prod_4','prod_5'], ['prod_11','prod_12','prod_15']]

# pdf_rec =pd.read_csv('/home/rijulizer/workspace/RecMet/src/TestData/recommendation_list.csv')
sdf_rec = spark.createDataFrame(pdf_rec)
# sdf_rec = sdf_rec.withColumn('Recommended',sf.array(sf.col('Recommended')))\
#                  .withColumn('Test',sf.array(sf.col('Test')))
print(sdf_rec.dtypes)
# sdf_rec.show()


[('User', 'bigint'), ('Recommended', 'array<string>'), ('Test', 'array<string>')]


In [7]:
# Randomly Initialise variables
weight_dict ={
    'prod_1':1,
    'prod_2':2,
    'prod_3':3,
    'prod_4':4,
    'prod_5':5,
    'prod_6':6,
    'prod_7':7,
    'prod_8':8,
    'prod_9':9,
    'prod_10':10,
    'prod_11':11,
    'prod_12':12,
    'prod_13':13,
    'prod_14':14,
    'prod_15':15,
    'prod_16':16,
    'prod_17':17,
    'prod_18':18,
}
weights =  weight_dict
longtail_thresh = 8


In [8]:
sdf_rec = sdf_rec.withColumn('TestLen',sf.size(sf.col('Test')))
sdf_rec.show()

+----+--------------------+--------------------+-------+
|User|         Recommended|                Test|TestLen|
+----+--------------------+--------------------+-------+
|   1|[prod_1, prod_2, ...|[prod_10, prod_4,...|      3|
|   2|[prod_10, prod_12...|[prod_1, prod_4, ...|      3|
|   3|[prod_11, prod_7,...|[prod_1, prod_2, ...|      9|
|   4|[prod_9, prod_12,...|           [prod_12]|      1|
+----+--------------------+--------------------+-------+



In [10]:
# # contains all the python codes
# import numpy as np

# class Metrics:
#     def __init__(self, weights, longtail_thresh):
#         self.weights = weights
#         self.longtail_thresh = longtail_thresh

#     # Average Intersection
#     def AI(self, recommended, actual):
        
#         common_num = len(list(set(actual).intersection(set(recommended))))
#         AI = round(common_num/ min(len(actual), len(recommended)), 4)
        
#         return AI

#     # Popularity Weighted Avergae Intersection PWAI
#     def PWAI(self, recommended, actual):
        
#         # Set the smaller set as denominator
#         if len(actual)<= len(recommended):
#             denom_elem = actual
#         else:
#             denom_elem = recommended
        
#         common_num_elem = list(set(actual).intersection(set(recommended)))
#         # weighted sum of numerator elements
#         num = np.array([self.weights[x] for x in common_num_elem]).sum()
#         denom = np.array([self.weights[x] for x in denom_elem]).sum()      

#         return float(round(num/denom, 4))
    
#     def ARP(self, recommended):
#         item_weights = np.array([self.weights[x] for x in recommended])
#         arp = item_weights.mean()

#         return float(round(arp, 4))
    
#     def APLT(self, recommended):
#         item_weights = np.array([self.weights[x] for x in recommended])
#         avg_longtail_items = (item_weights > self.longtail_thresh).astype(int).mean()
        
#         return float(round(avg_longtail_items, 4))

#     def ACLT(self, recommended):
#         item_weights = np.array([self.weights[x] for x in recommended])
#         sum_longtail_items = (item_weights > self.longtail_thresh).astype(int).sum()
        
#         return float(round(sum_longtail_items, 4))


In [11]:
# from PythonMetrics import Metrics
# # met = Metrics(weights, longtail_thresh)
# class SuperRecMet(Metrics):
#     def __init__(self, weights, longtail_thresh):
#         self.weights = weights
#         self.longtail_thresh = longtail_thresh
#         super().__init__(self.weights, self.longtail_thresh)
        
        
#         met = Metrics(self.weights, self.longtail_thresh)

#         # Pyspark AI
#         self.AI = sf.udf(lambda x, y: met.AI(x, y), FloatType()) 
        
#         # Pyspark PWAI
#         self.PWAI = sf.udf(lambda x, y: met.PWAI(x, y), FloatType())

#         # Pyspark ARP
#         self.ARP = sf.udf(lambda x: met.ARP(x), FloatType())

#         # Pyspark APLT
#         self.APLT = sf.udf(lambda x: met.APLT(x), FloatType())

#         # Pyspark ACLT
#         self.ACLT = sf.udf(lambda x: met.ACLT(x), FloatType())


In [9]:
from PythonMetrics import Metrics

# met = Metrics(weights, longtail_thresh)
class RecMet:
    def __init__(self, weights, longtail_thresh):
        
        self.weights = weights
        self.longtail_thresh = longtail_thresh
        # from PythonMetrics import Metrics
        met = Metrics(self.weights, self.longtail_thresh)

        # Pyspark AI
        self.AI = sf.udf(lambda x, y: met.AI(x, y), FloatType()) 
        
        # Pyspark PWAI
        self.PWAI = sf.udf(lambda x, y: met.PWAI(x, y), FloatType())

        # Pyspark ARP
        self.ARP = sf.udf(lambda x: met.ARP(x), FloatType())

        # Pyspark APLT
        self.APLT = sf.udf(lambda x: met.APLT(x), FloatType())

        # Pyspark ACLT
        self.ACLT = sf.udf(lambda x: met.ACLT(x), FloatType())


In [10]:
rm = RecMet(weights,longtail_thresh)

In [11]:
sdf_rec_metric= sdf_rec.withColumn('AI',rm.AI(sf.col('Recommended'), sf.col('Test')))\
    .withColumn('PWAI',rm.PWAI(sf.col('Recommended'), sf.col('Test')))\
    .withColumn('ARP',rm.ARP(sf.col('Recommended')))\
    .withColumn('APLT',rm.APLT(sf.col('Recommended')))\
    .withColumn('ACLT',rm.ACLT(sf.col('Recommended')))
sdf_rec_metric.show()

+----+--------------------+--------------------+-------+------+------+----+----+----+
|User|         Recommended|                Test|TestLen|    AI|  PWAI| ARP|APLT|ACLT|
+----+--------------------+--------------------+-------+------+------+----+----+----+
|   1|[prod_1, prod_2, ...|[prod_10, prod_4,...|      3|0.6667|0.4737| 3.0| 0.0| 0.0|
|   2|[prod_10, prod_12...|[prod_1, prod_4, ...|      3|0.6667|  0.95| 8.8| 0.6| 3.0|
|   3|[prod_11, prod_7,...|[prod_1, prod_2, ...|      9|   0.8|  0.84|10.0| 0.6| 3.0|
|   4|[prod_9, prod_12,...|           [prod_12]|      1|   1.0|   1.0|12.6| 1.0| 5.0|
+----+--------------------+--------------------+-------+------+------+----+----+----+



In [30]:
sdf_rec_metric.toPandas().head()

,User,Recommended,Test,AI,PWAI,ARP,APLT,ACLT
0,1,"[prod_1, prod_2, prod_3, prod_4, prod_5]","[prod_10, prod_4, prod_5]",0.6667,0.4737,3.0,0.0,0.0
1,2,"[prod_10, prod_12, prod_3, prod_4, prod_15]","[prod_1, prod_4, prod_15]",0.6667,0.9500,8.8,0.6,3.0
2,3,"[prod_11, prod_7, prod_8, prod_9, prod_15]","[prod_1, prod_2, prod_3, prod_7, prod_9, prod_...",0.8000,0.8400,10.0,0.6,3.0
3,4,"[prod_9, prod_12, prod_13, prod_14, prod_15]",[prod_12],1.0000,1.0000,12.6,1.0,5.0
